## Gene body counts

In [8]:
import numpy as np
import os
import sys
import collections
import matplotlib.pyplot as plt
import gzip
import loompy
# import scipy.sparse as sparse
import urllib.request
import pybedtools
from pybedtools import BedTool
import warnings
from sklearn.neighbors import NearestNeighbors
from matplotlib.collections import LineCollection

sys.path.append('/home/camiel/chromograph/')
# from chromograph.plotting.UMI_plot import UMI_plot
import chromograph

from umap import UMAP
import sklearn.metrics
from scipy.spatial import distance
import community
import networkx as nx
from scipy import sparse
from typing import *

In [5]:
## Import path to the relevant 10X reference dataset

ref = '/data/ref/cellranger-atac/refdata-cellranger-atac-GRCh38-1.1.0'
indir = '/data/proj/scATAC/chromograph/mouse_test2/'
f = os.path.join(indir, '10X_test_10kb.loom')

In [6]:
## Connect to loompy session
ds = loompy.connect(f)
print(ds.shape)

(271145, 4273)


In [9]:
genes = BedTool(os.path.join(ref, 'regions', 'transcripts.bed'))
genes.head()

chr1	11868	14409	DDX11L1	.	+	transcribed_unprocessed_pseudogene
 chr1	12009	13670	DDX11L1	.	+	transcribed_unprocessed_pseudogene
 chr1	14403	29570	WASH7P	.	-	unprocessed_pseudogene
 chr1	17368	17436	MIR6859-1	.	-	miRNA
 chr1	29553	31097	RP11-34P13.3	.	+	lincRNA
 chr1	30266	31109	RP11-34P13.3	.	+	lincRNA
 chr1	30365	30503	MIR1302-2	.	+	miRNA
 chr1	34553	36081	FAM138A	.	-	lincRNA
 chr1	52472	53312	OR4G4P	.	+	unprocessed_pseudogene
 chr1	57597	64116	OR4G11P	.	+	transcribed_unprocessed_pseudogene
 

In [25]:
coding = []

for x in genes:
    if x[6] == 'protein_coding':
        coding.append(x)

coding = BedTool(coding)
coding.head()

chr1	65418	71585	OR4F5	.	+	protein_coding
 chr1	69054	70108	OR4F5	.	+	protein_coding
 chr1	450702	451697	OR4F29	.	-	protein_coding
 chr1	685678	686673	OR4F16	.	-	protein_coding
 chr1	925737	944575	SAMD11	.	+	protein_coding
 chr1	925740	944581	SAMD11	.	+	protein_coding
 chr1	944203	959290	NOC2L	.	-	protein_coding
 chr1	960586	965715	KLHL17	.	+	protein_coding
 chr1	960638	965602	KLHL17	.	+	protein_coding
 chr1	966496	975108	PLEKHN1	.	+	protein_coding
 

In [13]:
feature = []
for x in genes:
    feature.append(x[6])
    
print(np.unique(feature))

['3prime_overlapping_ncRNA' 'IG_C_gene' 'IG_C_pseudogene' 'IG_D_gene'
 'IG_J_gene' 'IG_J_pseudogene' 'IG_V_gene' 'IG_V_pseudogene'
 'IG_pseudogene' 'Mt_rRNA' 'Mt_tRNA' 'TEC' 'TR_C_gene' 'TR_D_gene'
 'TR_J_gene' 'TR_J_pseudogene' 'TR_V_gene' 'TR_V_pseudogene' 'antisense'
 'bidirectional_promoter_lncRNA' 'lincRNA' 'macro_lncRNA' 'miRNA'
 'misc_RNA' 'non_coding' 'polymorphic_pseudogene' 'processed_pseudogene'
 'processed_transcript' 'protein_coding' 'pseudogene' 'rRNA' 'ribozyme'
 'sRNA' 'scRNA' 'scaRNA' 'sense_intronic' 'sense_overlapping' 'snRNA'
 'snoRNA' 'transcribed_processed_pseudogene'
 'transcribed_unitary_pseudogene' 'transcribed_unprocessed_pseudogene'
 'translated_processed_pseudogene' 'unitary_pseudogene'
 'unprocessed_pseudogene' 'vaultRNA']


In [ ]:
def count_fragments(frag_dict, barcodes, bsize):
    '''
    '''
    
    Count_dict = collections.OrderedDict()

    for bar in barcodes:    
        if bar in frag_dict:
            frags = frag_dict[bar]
            counts = {}
            for _frag in frags:

                # If a fragment spans two bins we count it twice
                for x in set([int(_frag[1]/bsize)*bsize+1, int(_frag[2]/bsize)*bsize+1]):
                    k = (_frag[0], x, x + bsize - 1)
                    if k not in counts.keys():
                        counts[k] = 1
                    else:
                        counts[k] += 1
            Count_dict[bar] = counts
        else:
            continue
    
    return Count_dict;
